# Tutorial 4: Error handling

This tutorial explains how errors are reported by the zhmcclient package to its users, and how they can be handled.

Error handling in the zhmcclient package is based on Python exceptions.

If you have not dealt with Python exceptions yet, here are a few good articles about them:

* [Jeff Knupp: Write Cleaner Python: Use Exceptions](https://jeffknupp.com/blog/2013/02/06/write-cleaner-python-use-exceptions/)
* [Eli Bendersky: Robust exception handling](http://eli.thegreenplace.net/2008/08/21/robust-exception-handling/)
* [Ian Bicking: Re-raising Exceptions](http://www.ianbicking.org/blog/2007/09/re-raising-exceptions.html)
* [Sheena: Writing and Using Custom Exceptions in Python](https://www.codementor.io/python/tutorial/how-to-write-python-custom-exceptions)
* [Joel Spolsky: Exceptions](http://www.joelonsoftware.com/items/2003/10/13.html) (a critical view on exceptions)

The zhmcclient package defines a number of exceptions to indicate specific error situations at run time. A prominent example is a connection error in the communication with the HMC, or invalid input parameters for an HMC operation.

The zhmcclient exceptions are all derived from [`zhmcclient.Error`](https://python-zhmcclient.readthedocs.io/en/latest/general.html#zhmcclient.Error), in order to have a common base exception to catch.

The zhmcclient exceptions are provided in order to have a stable definition for users of what can legally go wrong; they do not cover everything that can possibly go wrong. Any exception that is raised by the zhmcclient package that is not derived from `zhmcclient.Error` should be considered a programming error.

The zhmcclient exceptions are:

* [`zhmcclient.ConnectionError`](https://python-zhmcclient.readthedocs.io/en/latest/general.html#zhmcclient.ConnectionError) - indicates a problem with the connection to the HMC, at the transport level or below.
* [`zhmcclient.AuthError`](https://python-zhmcclient.readthedocs.io/en/latest/general.html#zhmcclient.AuthError) - indicates an authentication error with the HMC, either at the TLS/SSL handshake level (e.g. with CA certificates), or at the HTTP level.
* [`zhmcclient.HTTPError`](https://python-zhmcclient.readthedocs.io/en/latest/general.html#zhmcclient.HTTPError) - indicates that the HMC returned an HTTP response with a bad HTTP status code.
* [`zhmcclient.ParseError`](https://python-zhmcclient.readthedocs.io/en/latest/general.html#zhmcclient.ParseError) - indicates a parsing error while processing a response from the HMC.
* [`zhmcclient.VersionError`](https://python-zhmcclient.readthedocs.io/en/latest/general.html#zhmcclient.VersionError) - indicates that there is a version mismatch between the HMC API versions supported by the client and by the HMC.

The zhmcclient documentation lists only the zhmcclient exceptions that can be raised. Other exceptions may in addition be raised and should be considered programming errors.

The following code can be used to trigger different exceptions:

* `zhmcclient.ConnectionError`, if the HMC cannot be reached on the network.
* `zhmcclient.AuthError`, if the HMC can be reached on the network, but the userid does not exist or the password is invalid.
* `zhmcclient.HTTPError`, if the password is the empty string, or if it is an integer instead of a string.
* Python `RuntimeError`, if you run this on a Python version that is not supported for the zhmcclient package (sorry, this was the only Python exception we could come up with for this particular piece of code).

In [ ]:
zhmc = '9.152.150.65'  # HMC IP address or host name
userid = 'foo'         # HMC userid
password= '123'        # HMC password

import zhmcclient
import requests
requests.packages.urllib3.disable_warnings()
try:
    session = zhmcclient.Session(zhmc, userid, password)
    session.logon()
except zhmcclient.Error as exc:
    print("%s: %s" % (exc.__class__.__name__, exc))

The code section above shows the basic approach for handling exceptions with the zhmcclient package:
* Handle all exceptions derived from `zhmcclient.Error`.
* Ignore all other exceptions (or let them be handled like you would handle programming errors).